In [129]:
import pandas as pd
import numpy as np

#viz
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
#classe pour l'ACP
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn import decomposition
from sklearn import preprocessing

#Model évaluation / ensemble models
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier

#eval
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
from sklearn.inspection import permutation_importance
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
from sklearn.inspection import permutation_importance

# Tuning
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats
from sklearn.svm import SVC

import warnings
warnings.filterwarnings("ignore")

In [90]:
df = pd.read_csv('data/data.csv')
#replace the column's names
df.columns = [c.replace(" ","", 1) if c.startswith(' ') else c for c in df.columns]

In [91]:
df

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
0,1,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,0.302646,...,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1,0.016469
1,1,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,0.303556,...,0.795297,0.008323,0.623652,0.610237,0.839969,0.283846,0.264577,0.570175,1,0.020794
2,1,0.426071,0.499019,0.472295,0.601450,0.601364,0.998857,0.796403,0.808388,0.302035,...,0.774670,0.040003,0.623841,0.601449,0.836774,0.290189,0.026555,0.563706,1,0.016474
3,1,0.399844,0.451265,0.457733,0.583541,0.583541,0.998700,0.796967,0.808966,0.303350,...,0.739555,0.003252,0.622929,0.583538,0.834697,0.281721,0.026697,0.564663,1,0.023982
4,1,0.465022,0.538432,0.522298,0.598783,0.598783,0.998973,0.797366,0.809304,0.303475,...,0.795016,0.003878,0.623521,0.598782,0.839973,0.278514,0.024752,0.575617,1,0.035490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6814,0,0.493687,0.539468,0.543230,0.604455,0.604462,0.998992,0.797409,0.809331,0.303510,...,0.799927,0.000466,0.623620,0.604455,0.840359,0.279606,0.027064,0.566193,1,0.029890
6815,0,0.475162,0.538269,0.524172,0.598308,0.598308,0.998992,0.797414,0.809327,0.303520,...,0.799748,0.001959,0.623931,0.598306,0.840306,0.278132,0.027009,0.566018,1,0.038284
6816,0,0.472725,0.533744,0.520638,0.610444,0.610213,0.998984,0.797401,0.809317,0.303512,...,0.797778,0.002840,0.624156,0.610441,0.840138,0.275789,0.026791,0.565158,1,0.097649
6817,0,0.506264,0.559911,0.554045,0.607850,0.607850,0.999074,0.797500,0.809399,0.303498,...,0.811808,0.002837,0.623957,0.607846,0.841084,0.277547,0.026822,0.565302,1,0.044009


In [92]:
# définition de la commande
pca = PCA(n_components=6)

# Sélection des variables quantitatives
dat= df.drop(df[['Bankrupt?','Cash flow rate', 'Net Income Flag']], axis=1)

# réduction
data = pd.DataFrame(scale(dat),columns=dat.columns)

# composantes principales
C = pca.fit(data).transform(data)
C

array([[-7.32931539,  0.41936902,  0.40041744, -1.09574889, -0.16433452,
         1.40364256],
       [-2.69865528, -0.96392398,  2.72180332, -1.82731651, -0.83602974,
         0.68585931],
       [-4.27454422, -0.38874328,  0.47619539, -0.63483945, -0.33753341,
        -0.15693158],
       ...,
       [ 1.87863693, -1.69126372,  1.25423769,  1.48769928,  0.08102592,
        -1.37757209],
       [ 1.78759102, -0.38154176,  0.21896999,  0.75157925,  0.04335248,
         0.0911847 ],
       [ 1.52988914, -2.42501456,  0.36960562,  3.42750012,  0.13099008,
        -2.12298945]])

In [93]:
df_pca = pd.DataFrame(C)

In [94]:
df_pca[['Bankrupt?']] = df[['Bankrupt?']] #, 'Cash flow rate', 'Net Income Flag'

In [95]:
df_pca

,0,1,2,3,4,5,Bankrupt?
0,-7.329315,0.419369,0.400417,-1.095749,-0.164335,1.403643,1
1,-2.698655,-0.963924,2.721803,-1.827317,-0.836030,0.685859,1
2,-4.274544,-0.388743,0.476195,-0.634839,-0.337533,-0.156932,1
3,-5.877391,-1.332273,1.658755,-1.932723,-0.732798,0.105283,1
4,-2.320097,-0.201739,-0.597387,-1.151328,-0.204844,-0.258886,1
...,...,...,...,...,...,...,...
6814,-1.160183,-0.508168,-0.230724,0.195690,-0.063851,-0.070507,0
6815,-1.245241,-0.672827,-0.065996,-0.431402,-0.231313,-0.682978,0
6816,1.878637,-1.691264,1.254238,1.487699,0.081026,-1.377572,0
6817,1.787591,-0.381542,0.218970,0.751579,0.043352,0.091185,0


In [120]:
y= df_pca['Bankrupt?']
X= df_pca.drop('Bankrupt?', axis=1)

In [121]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state=0)

In [122]:
X_train

,0,1,2,3,4,5
4106,3.115270,-0.490652,-0.588281,2.245240,0.425237,0.612788
2799,3.414949,1.796891,-0.497805,-1.609490,-0.003848,0.379826
1596,1.172850,-3.583547,2.868992,2.613096,0.009978,-1.355265
2278,-5.779766,2.688762,-2.887427,-0.151390,0.156672,-1.216198
1493,4.305346,-1.549334,0.107752,3.921579,0.665727,-0.402907
...,...,...,...,...,...,...
4931,-2.072350,-1.305927,1.637048,-0.606787,-0.533789,-0.218551
3264,4.878909,0.458307,0.908123,0.421710,0.105388,0.735453
1653,2.289113,-0.217629,1.400008,-1.059683,-0.336702,-0.723494
2607,1.515350,-2.137234,0.734297,2.108094,0.123953,-1.549805


In [123]:
y_train

4106    0
2799    0
1596    0
2278    0
1493    0
       ..
4931    0
3264    0
1653    0
2607    0
2732    0
Name: Bankrupt?, Length: 4773, dtype: int64

## Model

In [130]:
model_1 = SGDClassifier(random_state=0)
model_2 = DecisionTreeClassifier(random_state=0)
model_3 = KNeighborsClassifier(n_neighbors=2)
model_4 = RandomForestClassifier(n_estimators=100)
model_5 = GradientBoostingClassifier(n_estimators=100)

model_6 = VotingClassifier([('SGD', model_1),
                           ('Tree', model_2),
                           ('KNN', model_3),
                            ('Forest', model_4),
                           ('GBC', model_5)],
                           voting='hard')

for model in (model_1, model_2, model_3, model_4, model_5, model_6):
    model.fit(X_train, y_train)
    print(model.__class__.__name__, model.score(X_test, y_test))

SGDClassifier 0.9672531769305963
DecisionTreeClassifier 0.9467253176930597
KNeighborsClassifier 0.9716520039100685
RandomForestClassifier 0.9701857282502444
GradientBoostingClassifier 0.9667644183773216
VotingClassifier 0.9711632453567938


In [128]:
model = BaggingClassifier(base_estimator=KNeighborsClassifier(),
                         n_estimators=100)

model.fit(X_train, y_train)
model.score(X_test, y_test)

0.9701857282502444